# Downloading Necessary Libraries

In [ ]:
!pip install opencv-python-headless requests ultralytics pandas schedule sqlite-utils

# Imports

In [ ]:
import cv2
import time
import os
import requests
import sqlite3
from datetime import datetime, timedelta
from ultralytics import YOLO
from queue import Queue
from threading import Thread, Lock

# Helper Functions and Variables

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
frame_queue = Queue()

In [ ]:
db_file = "vehicle_counts.db"
db_lock = Lock()

In [ ]:
def init_db():
    """Initialize the SQLite database."""
    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS vehicle_counts (
                latitude REAL,
                longitude REAL,
                timestamp TEXT,
                count INTEGER,
                PRIMARY KEY (latitude, longitude, timestamp)
            )
        """)
        conn.commit()

In [ ]:
def fetch_camera_feed(api_key):
    api_url = "https://511ny.org/api/getcameras"
    params = {
        'key': api_key,
        'format': 'json'
    }
    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        cameras = response.json()

        congestion_zone_cameras = []
        for camera in cameras:
            url = camera.get('Url')
            lat = camera.get('Latitude')
            lon = camera.get('Longitude')

            if url and lat and lon:
                if 40.7000 <= lat <= 40.7680 and -74.0100 <= lon <= -73.9665:
                    congestion_zone_cameras.append({"Url": url, "Latitude": lat, "Longitude": lon})

        return congestion_zone_cameras

    except requests.RequestException as e:
        print("Error fetching camera feeds:", {e})
        return []

In [ ]:
def process_frame(frame, latitude, longitude, timestamp):
    results = model(frame)
    detections = results[0].boxes.data  

    vehicle_classes = [2, 3, 5, 7] 
    vehicle_count = sum(1 for detection in detections if int(detection[5]) in vehicle_classes)

    with db_lock, sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO vehicle_counts (latitude, longitude, timestamp, count)
            VALUES (?, ?, ?, ?)
            ON CONFLICT(latitude, longitude, timestamp) DO UPDATE SET count = count + ?
        """, (latitude, longitude, timestamp, vehicle_count, vehicle_count))
        conn.commit()

In [ ]:
def capture_frames(feed_url, latitude, longitude, duration):
    cap = cv2.VideoCapture(feed_url)
    if not cap.isOpened():
        return

    start_time = time.time()
    while time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            break

        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
        frame_queue.put((frame, latitude, longitude, timestamp))

    cap.release()

In [ ]:
def frame_processing_worker():
    while True:
        frame, lat, lon, timestamp = frame_queue.get()
        process_frame(frame, lat, lon, timestamp)
        frame_queue.task_done()

In [ ]:
def run_pipeline(duration_hours=12):
    API_KEY = os.getenv("API_KEY")  
    
    duration_per_iteration = 60 
    end_time = datetime.now() + timedelta(hours=duration_hours)

    cameras = fetch_camera_feed(API_KEY)

    while datetime.now() < end_time:
        for camera in cameras:
            capture_frames(camera["Url"], camera["Latitude"], camera["Longitude"], duration_per_iteration)
        time.sleep(duration_per_iteration)

    print(f"{duration_hours}-hour processing completed.")

# Data Collection - Main Code/ Function

In [ ]:
init_db()

In [ ]:
for _ in range(4):
    Thread(target=frame_processing_worker, daemon=True).start()

In [ ]:
run_pipeline(duration_hours=12)

# Converting Latitude-Longitude to Street Name

In [ ]:
import sqlite3
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from time import sleep

In [ ]:
def get_street_name(lat, lon, retries=3):
    for _ in range(retries):
        try:
            location = geolocator.reverse((lat, lon), exactly_one=True)
            if location and 'road' in location.raw['address']:
                return location.raw['address']['road']
            return "Unknown Street"
        except GeocoderTimedOut:
            sleep(1) 
        except Exception as e:
            print("Error retrieving street name", {e})
            return "Error"
    return "Error"

In [ ]:
db_path = "vehicle_counts.db" 
conn = sqlite3.connect(db_path)

In [ ]:
vehicle_counts_data = pd.read_sql_query("SELECT * FROM vehicle_counts LIMIT 5000;", conn)
conn.close()

In [ ]:
geolocator = Nominatim(user_agent="reverse_geocoder", timeout=10)

In [ ]:
vehicle_counts_data['street_name'] = vehicle_counts_data.apply(
    lambda row: get_street_name(row['latitude'], row['longitude']), axis=1
)

In [ ]:
output_db_path = "vehicle_counts_with_streets.db"
conn_new = sqlite3.connect(output_db_path)

In [ ]:
vehicle_counts_data.to_sql("vehicle_counts_with_streets", conn_new, if_exists="replace", index=False)
conn_new.close()